In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import re
import nltk
from nltk import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

### 데이터 확인

In [2]:
# 파일 불러오기

train = pd.read_csv('open/train.csv', encoding='utf-8')
test_x = pd.read_csv('open/test_x.csv', encoding='utf-8')
submission = pd.read_csv('open/sample_submission.csv', encoding='utf-8')

In [3]:
# 데이터 확인

train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [4]:
train.author.unique(), train.author.value_counts()

(array([3, 2, 1, 4, 0]),
 3    15063
 0    13235
 2    11554
 4     7805
 1     7222
 Name: author, dtype: int64)

In [5]:
test_x

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [18]:
test_x['author'] = ''

### 전처리

In [7]:
def text_(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text'] = train['text'].apply(text_)
test_x['text'] = test_x['text'].apply(text_)

In [50]:
train['text'] = train['text'].str.lower()
test_x['text'] = test_x['text'].str.lower()

In [51]:
train

,index,text,author
0,0,he was almost choking there was so much so muc...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in peru...,1
3,3,the captain was in the porch keeping himself c...,4
4,4,have mercy gentlemen odin flung up his hands d...,3
...,...,...,...
54874,54874,is that you mr smith odin whispered i hardly d...,2
54875,54875,i told my plan to the captain and between us w...,4
54876,54876,your sincere wellwisher friend and sister luc...,1
54877,54877,then you wanted me to lend you money,3


In [32]:
test_x

,index,text,author
0,0,“Not at all. I think she is one of the most ch...,
1,1,"""No,"" replied he, with sudden consciousness, ""...",
2,2,As the lady had stated her intention of scream...,
3,3,“And then suddenly in the silence I heard a so...,
4,4,His conviction remained unchanged. So far as I...,
...,...,...,...
19612,19612,"At the end of another day or two, odin growing...",
19613,19613,"All afternoon we sat together, mostly in silen...",
19614,19614,"odin, having carried his thanks to odin, proc...",
19615,19615,"Soon after this, upon odin's leaving the room,...",


In [6]:
train.dtypes

index      int64
text      object
author     int64
dtype: object

In [23]:
X_train = train.loc[:, 'text']

In [24]:
y_train = train.loc[:, 'author']

In [21]:
X_test = test_x.loc[:, 'text']

In [121]:
y_test = test_x.loc[:, 'author']

In [118]:
test_x.loc[:, 'author'] = np.random.randint(0, 6)

In [120]:
test_x.loc[:, 'author'].value_counts()

1    19617
Name: author, dtype: int64

In [113]:
np.random.randint(0, 6)

1

In [90]:
# X = list(map(str, train.loc[:, 'text']))
# len(X)

In [56]:
# y = list(train.loc[:, 'author'])
# len(y)

54879

In [12]:
# y = list(map(str, y))

In [49]:
# Train = list(zip(X, y))
# Train

NameError: name 'X' is not defined

#### 문장 토큰화

In [25]:
from nltk.tokenize import sent_tokenize

print(sent_tokenize(X[0]))

['He was almost choking.', 'There was so much, so much he wanted to say, but strange exclamations were all that came from his lips.', 'The Pole gazed fixedly at him, at the bundle of notes in his hand; looked at odin, and was in evident perplexity.']


#### 벡터라이즈

##### countvectorizer

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [27]:
ftr_vect = cnt_vect.transform(X_train)
print('X_train의 CountVectorizer Shape:', ftr_vect.shape)

X_train의 CountVectorizer Shape: (54879, 34719)


In [28]:
print(cnt_vect.vocabulary_)

{'he': 14720, 'was': 33645, 'almost': 1774, 'choking': 5986, 'there': 30748, 'so': 28390, 'much': 20270, 'wanted': 33577, 'to': 31113, 'say': 26733, 'but': 4963, 'strange': 29453, 'exclamations': 11400, 'were': 33860, 'all': 1716, 'that': 30698, 'came': 5135, 'from': 13081, 'his': 15069, 'lips': 18329, 'the': 30705, 'pole': 23309, 'gazed': 13419, 'fixedly': 12342, 'at': 2680, 'him': 15037, 'bundle': 4873, 'of': 21275, 'notes': 20934, 'in': 15957, 'hand': 14452, 'looked': 18514, 'odin': 21210, 'and': 1965, 'evident': 11296, 'perplexity': 22625, 'your': 34643, 'sister': 27938, 'asked': 2546, 'for': 12683, 'it': 16995, 'suppose': 29951, 'she': 27458, 'engaged': 10877, 'one': 21377, 'day': 8373, 'as': 2516, 'walked': 33529, 'perusing': 22700, 'jane': 17064, 'last': 17837, 'letter': 18122, 'dwelling': 10288, 'on': 21372, 'some': 28521, 'passages': 22186, 'which': 33952, 'proved': 24191, 'had': 14358, 'not': 20920, 'written': 34504, 'spirits': 28815, 'when': 33926, 'instead': 16569, 'being':

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(ftr_vect, y)
pred = lr_clf.predict(ftr_vect)

In [31]:
print(accuracy_score(y_test, pred))

ValueError: Found input variables with inconsistent numbers of samples: [19617, 54879]

In [80]:
# stop_words, 'english' 추가

In [19]:
cnt_vect = CountVectorizer(max_features=5, stop_words='english')
cnt_vect.fit(X)
ftr_vect = cnt_vect.transform(X)
print(type(ftr_vect), ftr_vect.shape)
print(cnt_vect.vocabulary_)

<class 'scipy.sparse.csr.csr_matrix'> (54879, 5)
{'odin': 3, 'mr': 2, 'said': 4, 'man': 1, 'know': 0}


In [ ]:
# n gram 확인 -- 노트북 사양으로 확인 불가

In [85]:
cnt_vect = CountVectorizer(ngram_range=(2,2))
cnt_vect.fit(X)
ftr_vect = cnt_vect.transform(X)
print(type(ftr_vect), ftr_vect.shape)
print(cnt_vect.vocabulary_)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



##### tfidfvectorizer

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect = TfidfVectorizer(stop_words='english')
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

In [102]:
tfid_X = tfid_vect.transform(X)
tfid_X

<54879x46917 sparse matrix of type '<class 'numpy.float64'>'
	with 859853 stored elements in Compressed Sparse Row format>

In [104]:
print(type(tfid_X), tfid_X.shape)

<class 'scipy.sparse.csr.csr_matrix'> (54879, 46917)


In [106]:
tfid_vect.vocabulary_

{'choking': 6933,
 'wanted': 44886,
 'say': 35464,
 'strange': 39369,
 'exclamations': 13816,
 'came': 5783,
 'lips': 23969,
 'pole': 31170,
 'gazed': 16698,
 'fixedly': 15217,
 'bundle': 5435,
 'notes': 27723,
 'hand': 18261,
 'looked': 24265,
 'odin': 28167,
 'evident': 13688,
 'perplexity': 30305,
 'sister': 37306,
 'asked': 2359,
 'suppose': 40019,
 'engaged': 13162,
 'day': 9963,
 'walked': 44816,
 'perusing': 30396,
 'janes': 22167,
 'letter': 23625,
 'dwelling': 12444,
 'passages': 29747,
 'proved': 32275,
 'jane': 22160,
 'written': 46484,
 'spirits': 38554,
 'instead': 21405,
 'surprised': 40105,
 'mr': 26745,
 'saw': 35448,
 'looking': 24275,
 'meeting': 25528,
 'putting': 32544,
 'away': 2824,
 'immediately': 20450,
 'forcing': 15686,
 'smile': 37722,
 'said': 35151,
 'captain': 5966,
 'porch': 31308,
 'keeping': 22582,
 'carefully': 6037,
 'way': 45158,
 'treacherous': 42351,
 'shot': 36845,
 'intended': 21490,
 'turned': 42778,
 'spoke': 38607,
 'doctors': 11694,
 'watch':

In [107]:
#############################################

In [18]:
# 전체 말뭉치

all_words = set(word.lower() for sentence in X for word in word_tokenize(sentence))
all_words

{'westwhom',
 'abstracted',
 'numbered',
 'shunned',
 'canted',
 'venetians',
 'perverted',
 'impetuosity',
 'agitatedvery',
 'creatureyou',
 'presses',
 'otherthats',
 'jurymenfour',
 'shoving',
 'bun',
 'thunder',
 'customer',
 'affectionate',
 'yesratheri',
 'thepie',
 'ishes',
 'perceptive',
 'brothers',
 'quarrels',
 'selfsatisfaction',
 'model',
 'crystallisation',
 'holdenand',
 'attorney',
 'broughtup',
 'wasfor',
 'unkind',
 'alpinestock',
 'suharev',
 'marker',
 'greasewhich',
 'simpson',
 'realitya',
 'armchairs',
 'goodhearted',
 'badge',
 'vantageground',
 'girlwhat',
 'bounty',
 'wholehearted',
 'deputed',
 'potkins',
 'hereit',
 'creepers',
 'poora',
 'trailing',
 'vodincourt',
 'brewerylane',
 'bluchers',
 'illwillers',
 'concealmentodin',
 'uncontradicted',
 'objectit',
 'wonderments',
 'aloudmarthas',
 'detail',
 'boymy',
 'redemption',
 'has',
 'wordmiss',
 'flymore',
 'manure',
 'whipcorrected',
 'ee',
 'handkerchiefodins',
 'dromedary',
 'youthyouth',
 'illtaste',


In [19]:
# 다시 말뭉치

all_words = []
for sentence in X:
    for word in word_tokenize(sentence):
        all_words.append(word.lower())
        
set(all_words)

{'westwhom',
 'abstracted',
 'numbered',
 'shunned',
 'canted',
 'venetians',
 'perverted',
 'impetuosity',
 'agitatedvery',
 'creatureyou',
 'presses',
 'otherthats',
 'jurymenfour',
 'shoving',
 'bun',
 'thunder',
 'customer',
 'affectionate',
 'yesratheri',
 'thepie',
 'ishes',
 'perceptive',
 'brothers',
 'quarrels',
 'selfsatisfaction',
 'model',
 'crystallisation',
 'holdenand',
 'attorney',
 'broughtup',
 'wasfor',
 'unkind',
 'alpinestock',
 'suharev',
 'marker',
 'greasewhich',
 'simpson',
 'realitya',
 'armchairs',
 'goodhearted',
 'badge',
 'vantageground',
 'girlwhat',
 'bounty',
 'wholehearted',
 'deputed',
 'potkins',
 'hereit',
 'creepers',
 'poora',
 'trailing',
 'vodincourt',
 'brewerylane',
 'bluchers',
 'illwillers',
 'concealmentodin',
 'uncontradicted',
 'objectit',
 'wonderments',
 'aloudmarthas',
 'detail',
 'boymy',
 'redemption',
 'has',
 'wordmiss',
 'flymore',
 'manure',
 'whipcorrected',
 'ee',
 'handkerchiefodins',
 'dromedary',
 'youthyouth',
 'illtaste',


In [21]:
# 훈련용 데이터에 마킹

t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in Train[:10]]
t

KeyboardInterrupt: 

In [ ]:
# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제

import re
text = "I was wondering if anyone out there could enlighten me on this car."
shortword = re.compile(r'\W*\b\w{1,2}\b')
print(shortword.sub('', text))

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [30]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(X)
print(X_train_dtm.shape)

(54879, 47221)


In [33]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(54879, 47221)


In [23]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [34]:
mod = MultinomialNB()
mod.fit(tfidfv, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# 나이브베이즈를 활용한 유사 문서 검색

In [39]:
pd.Series(y).value_counts()

3    15063
0    13235
2    11554
4     7805
1     7222
dtype: int64

In [61]:
target = np.array(y)
target[:10]

array([3, 2, 1, 4, 3, 4, 3, 2, 0, 4])

In [45]:
# CountVectorize

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape

(54879, 47221)

In [48]:
# 몇 개 안되는 1의 갯수

import numpy as np

np.sum(X_train_counts.toarray()[0])

46

In [50]:
# Tfidf 적용

from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(54879, 47221)

In [62]:
# Multinomial Naive Bayes 적용

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, target)

In [55]:
test_x.text

0        not at all i think she is one of the most char...
1        no replied he with sudden consciousness not to...
2        as the lady had stated her intention of scream...
3        and then suddenly in the silence i heard a sou...
4        his conviction remained unchanged so far as i ...
                               ...                        
19612    at the end of another day or two odin growing ...
19613    all afternoon we sat together mostly in silenc...
19614     odin having carried his thanks to odin procee...
19615    soon after this upon odins leaving the room ma...
19616    and all the worse for the doomed man that the ...
Name: text, Length: 19617, dtype: object

In [56]:
test_x.text[0]

'not at all i think she is one of the most charming young ladies i ever met and might have been most useful in such work as we have been doing she had a decided genius that way witness the way in which she preserved that agra plan from all the other papers of her father but love is an emotional thing and whatever is emotional is opposed to that true cold reason which i place above all things i should never marry myself lest i bias my judgment'

In [63]:
# 간편 테스트

docs_new = [test_x.text[0], test_x.text[1]]
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, author in zip(docs_new, predicted):
    print('%r => %s' % (doc, target))

'not at all i think she is one of the most charming young ladies i ever met and might have been most useful in such work as we have been doing she had a decided genius that way witness the way in which she preserved that agra plan from all the other papers of her father but love is an emotional thing and whatever is emotional is opposed to that true cold reason which i place above all things i should never marry myself lest i bias my judgment' => [3 2 1 ... 1 3 0]
'no replied he with sudden consciousness not to find it in you for i cannot be ignorant that to you to your goodness i owe it alli feel iti would express it if i couldbut as you well know i am no orator' => [3 2 1 ... 1 3 0]
